In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("notebook26").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

23/09/26 17:02:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.csv("hdfs://namenode:9000/user/root/input/googleplaystore1.csv",escape='"', header=True)
#display(df)
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [3]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [5]:
df.count()

10841

## Data Cleaning Step

In [6]:
df = df.drop("size")

In [7]:
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [8]:
df = df.drop("Content Rating","Last Updated","Android Ver")

In [9]:
df.select('Category').distinct().show(34)

+-------------------+
|           Category|
+-------------------+
|             EVENTS|
|             COMICS|
|             SPORTS|
|            WEATHER|
|      VIDEO_PLAYERS|
|  AUTO_AND_VEHICLES|
|          PARENTING|
|      ENTERTAINMENT|
|    PERSONALIZATION|
| HEALTH_AND_FITNESS|
|   TRAVEL_AND_LOCAL|
|BOOKS_AND_REFERENCE|
|     FOOD_AND_DRINK|
|        PHOTOGRAPHY|
|           BUSINESS|
|             FAMILY|
|           SHOPPING|
|     HOUSE_AND_HOME|
|               GAME|
|                1.9|
|          EDUCATION|
|       PRODUCTIVITY|
|              TOOLS|
|          LIFESTYLE|
| NEWS_AND_MAGAZINES|
|            MEDICAL|
|      COMMUNICATION|
|MAPS_AND_NAVIGATION|
|             DATING|
| LIBRARIES_AND_DEMO|
|             BEAUTY|
|             SOCIAL|
|            FINANCE|
|     ART_AND_DESIGN|
+-------------------+



In [10]:
df = df.drop('Current Ver')

In [11]:
df.show(1,truncate=False)

+----------------------------------------------+--------------+------+-------+--------+----+-----+------------+
|App                                           |Category      |Rating|Reviews|Installs|Type|Price|Genres      |
+----------------------------------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |10,000+ |Free|0    |Art & Design|
+----------------------------------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [12]:
df = df.withColumn('Reviews',df.Reviews.cast(IntegerType()))\
       .withColumn('Installs', regexp_replace(col('Installs'),"[^0-9]",""))

In [13]:
df.show(2,truncate=False)

+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|App                                           |Category      |Rating|Reviews|Installs|Type|Price|Genres                   |
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |10000   |Free|0    |Art & Design             |
|Coloring book moana                           |ART_AND_DESIGN|3.9   |967    |500000  |Free|0    |Art & Design;Pretend Play|
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
only showing top 2 rows



In [14]:
df = df.withColumn('Installs', col('Installs').cast(IntegerType()))\
       .withColumn('Price',regexp_replace(col('Price'),"[$]",""))\
       .withColumn('Price',col('Price').cast(IntegerType()))

In [15]:
df.show(2, truncate=False)

+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|App                                           |Category      |Rating|Reviews|Installs|Type|Price|Genres                   |
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
|Photo Editor & Candy Camera & Grid & ScrapBook|ART_AND_DESIGN|4.1   |159    |10000   |Free|0    |Art & Design             |
|Coloring book moana                           |ART_AND_DESIGN|3.9   |967    |500000  |Free|0    |Art & Design;Pretend Play|
+----------------------------------------------+--------------+------+-------+--------+----+-----+-------------------------+
only showing top 2 rows



In [16]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



## Top reviews give to the apps

In [17]:
df.select('App').distinct().count()

9660

In [18]:
df1 = df.groupBy('App').agg(sum(col('Reviews')).alias('Reviews'))

In [19]:
df2 = df1.sort(df1.Reviews.desc())

### The Top 10 apps whose making good score in Reviews 

In [20]:
df2.show(10,truncate=False)

+----------------------------------------+---------+
|App                                     |Reviews  |
+----------------------------------------+---------+
|Instagram                               |266241989|
|WhatsApp Messenger                      |207348304|
|Clash of Clans                          |179558781|
|Messenger – Text and Video Chat for Free|169932272|
|Subway Surfers                          |166331958|
|Candy Crush Saga                        |156993136|
|Facebook                                |156286514|
|8 Ball Pool                             |99386198 |
|Clash Royale                            |92530298 |
|Snapchat                                |68045010 |
+----------------------------------------+---------+
only showing top 10 rows



## The Top 10 apps whose have a gib number of  installs

In [21]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [22]:
df5 = df.groupBy('App').agg(sum('Installs').alias('Installs'))
df6 = df5.sort(df5.Installs.desc())
df6.show(10,truncate=False)

+----------------------------------------+----------+
|App                                     |Installs  |
+----------------------------------------+----------+
|Subway Surfers                          |6000000000|
|Google Drive                            |4000000000|
|Instagram                               |4000000000|
|Hangouts                                |4000000000|
|Google News                             |4000000000|
|Google Photos                           |4000000000|
|Candy Crush Saga                        |3500000000|
|Maps - Navigate & Explore               |3000000000|
|Google Chrome: Fast & Secure            |3000000000|
|Messenger – Text and Video Chat for Free|3000000000|
+----------------------------------------+----------+
only showing top 10 rows



In [23]:
df3 = df.groupBy('Category').agg(sum('Installs').alias('Installs'))

In [24]:
df4= df3.sort(df3.Installs.desc())

In [25]:
df4.show(10,truncate=False)

+------------------+-----------+
|Category          |Installs   |
+------------------+-----------+
|GAME              |35086024415|
|COMMUNICATION     |32647276251|
|PRODUCTIVITY      |14176091369|
|SOCIAL            |14069867902|
|TOOLS             |11452771915|
|FAMILY            |10258263505|
|PHOTOGRAPHY       |10088247655|
|NEWS_AND_MAGAZINES|7496317760 |
|TRAVEL_AND_LOCAL  |6868887146 |
|VIDEO_PLAYERS     |6222002720 |
+------------------+-----------+
only showing top 10 rows



In [27]:
df7 = df.groupBy("App").agg(sum('Price').alias('Price'))

In [28]:
df8 = df7.sort(df7.Price.desc())
df8.show(10, truncate=False)

+------------------------------+-----+
|App                           |Price|
+------------------------------+-----+
|I'm Rich - Trump Edition      |400  |
|I am rich (Most expensive app)|399  |
|I am Rich!                    |399  |
|I am Rich                     |399  |
|I am rich(premium)            |399  |
|I Am Rich Pro                 |399  |
|most expensive app (H)        |399  |
|I Am Rich Premium             |399  |
|I AM RICH PRO PLUS            |399  |
|I am Rich Plus                |399  |
+------------------------------+-----+
only showing top 10 rows



In [ ]:
df4.write.parquet('hdfs://namenode:9000/user/root/output', mode='overwrite', partitionBy='Category')

In [ ]:
df9 = spark.read.parquet('hdfs://namenode:9000/user/root/output/Category=GAME', header=True)

In [ ]:
df9.print